In [68]:
# pip install litellm

In [7]:
import numpy as numpy
import pandas as pd
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import DuckDuckGoSearchResults, DuckDuckGoSearchRun
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel, LiteLLMModel, tool

import smtplib


load_dotenv()

True

In [15]:
# openai_model = ChatOpenAI()
# openai_model.invoke('who is rohit sharme?')

openai_model = LiteLLMModel(model_id="gpt-4o")


In [16]:
@tool
def log_action_to_excel(action: str) -> str:
    """
    Appends a new log entry to an Excel file.

    This function checks for the existence of the Excel log file and appends a new row
    with a unique action ID, the description of the action performed, and the current timestamp.
    
    Args:
        action (str): A description of the action performed.        
    Returns:
        
    """
    
    log_file: str = "data/logs.xlsx"

    # Check if the log file already exists
    if os.path.exists(log_file):
        # Read the existing log entries
        df = pd.read_excel(log_file)
    else:
        # Create a new DataFrame with the required columns if the file doesn't exist
        df = pd.DataFrame(columns=["action_id", "action", "time_stamp"])
    
    # Generate a unique action_id by taking the max and adding 1, or starting at 1 if empty
    new_id = int(df["action_id"].max() + 1) if not df.empty else 1
    
    # Record the current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create a new log entry
    new_entry = {"action_id": new_id, "action": action, "time_stamp": timestamp}
    new_entry_df = pd.DataFrame([new_entry])
    
    # Append the new entry to the DataFrame
    df = pd.concat([df, new_entry_df], ignore_index=True)
    
    # Save the updated DataFrame back to the Excel file
    df.to_excel(log_file, index=False)
    
    return (f"Logged action with id {new_id}: {action}")


In [17]:
@tool
def check_item_availability(item_name: str,requested_count: int) -> str:
    """
    A tool to check the availability of an item in the warehouse.

    This function performs the following steps:
    1. Reads the shelves file ("data/shelves.xlsx") to extract the item_id associated with the given item_name.
    2. Reads the warehouse file ("data/warehouse.xlsx") to retrieve the available count for that item_id.
    3. Compares the available count to the requested_count and returns a message indicating the result.

    Args:
        item_name: The name of the item to look up.
        requested_count: The number of units requested.

    Returns:
        A string message indicating whether the item is available in the warehouse, along with the count,
        or an error message if the item cannot be found.
    """
    try:
        # Define file paths (adjust these paths as necessary)
        shelves_file = os.path.join("data", "Shelves.xlsx")
        warehouse_file = os.path.join("data", "Inventory.xlsx")
        
        # Read the shelves file to map item names to item_ids
        shelves_df = pd.read_excel(shelves_file, engine='openpyxl')
        # Perform a case-insensitive search for the item_name
        item_row = shelves_df[shelves_df['shelf_category'].str.lower() == item_name.lower()]
        if item_row.empty:
            return f"Item '{item_name}' not found in shelves."
        
        # Extract the item_id from the shelves file
        item_id = item_row.iloc[0]['item_id']
        
        # Read the warehouse file to get the available count for the item_id
        warehouse_df = pd.read_excel(warehouse_file, engine='openpyxl')
        warehouse_row = warehouse_df[warehouse_df['item_id'] == item_id]
        if warehouse_row.empty:
            return f"Item '{item_name}' (ID: {item_id}) not found in warehouse."
        
        available_count = int(warehouse_row.iloc[0]['quantity_in_warehouse'])
        
        # Compare available count with the requested count and return the result
        if available_count >= requested_count:
            return f"Item '{item_name}' (ID: {item_id}) is available. Warehouse count: {available_count}."
        else:
            return (f"Item '{item_name}' (ID: {item_id}) is available, but only {available_count} units are "
                    f"in the warehouse, which is less than the requested {requested_count}.")
    except Exception as e:
        return f"Error checking availability: {str(e)}"



In [18]:
@tool
def send_email(recipient_email: str, subject: str, body: str) -> str:
    """
    A tool to check the availability of an item in the warehouse.

    This function performs the following steps:
    1. Take the recipient_email and subject as input.
    2. Send an email to the recipient_email with the subject and body.

    Args:
        recipient_email: The email address of the recipient.
        subject: The subject of the email.
        body: The body of the email.

    Returns:
        A string message indicating whether the item is available in the warehouse, along with the count,
        or an error message if the item cannot be found.
    """
    try:
        # Fetch sender credentials from environment variables
        sender_email = os.getenv("SENDER_EMAIL")
        sender_password = os.getenv("SENDER_PASSWORD")
        if not sender_email or not sender_password:
            return "Error: Missing sender credentials in environment variables."

        # Create an SMTP session with Gmail
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.starttls()
            smtp.login(sender_email, sender_password)

            # Construct the raw email message with headers
            message = f"""Subject: {subject}

                        {body}
                        """
            smtp.sendmail(sender_email, recipient_email, message)

        return f"Email successfully sent to {recipient_email}."
    except Exception as e:
        return f"Failed to send email: {str(e)}"



In [ ]:
agent = CodeAgent(
    tools=[check_item_availability, log_action_to_excel],
    model=openai_model,
    additional_authorized_imports=["pandas", "numpy", "os"]
)

agent.run("Check the availability of peanut butter with 10 units if not then send email to any available worker and update the log file for each step you take")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Check the availability of peanut butter with 10 units and update the log file                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            To achieve this task, I need to first check the availability of peanut butter with 10 units using the 
`check_item_availability` tool. Once I get the availability status, I will log this action using the 
`log_action_to_excel` tool.

Let's start by checking the availability of peanut butter with 10 units.
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 1.37 seconds| Input tokens: 2,211 | Output tokens: 66]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The availability of peanut butter with 10 units has been successfully checked and logged.")        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The availability of peanut butter with 10 units has been successfully checked and logged.

[Step 5: Duration 1.12 seconds| Input tokens: 13,347 | Output tokens: 358]

'The availability of peanut butter with 10 units has been successfully checked and logged.'

In [116]:
import smtplib
# creates SMTP session
s = smtplib.SMTP('smtp.gmail.com', 587)
# start TLS for security
s.starttls()
# Authentication
s.login("sathwik238@gmail.com", "oqtgklpdsqegvuph")
# message to be sent
message = "Message_you_need_to_send"
# sending the mail
s.sendmail("sathwik238@gmail.com", "sathwikmi45@gmail.com", message)
# terminating the session
s.quit()

(221,
 b'2.0.0 closing connection af79cd13be357-7c5b93485f7sm390342385a.74 - gsmtp')